In [ ]:
#| default_exp datamodels

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from dataclasses import dataclass, asdict, fields
from pathlib import Path
from fastcore.test import test_fail

@dataclass
class BaseDataClass:
    def to_dict(self):
        """Returning contents of the dataclass as a dictionary."""
        return asdict(self)

    @classmethod
    def from_dict(cls, **params):
        """Creating dataclass from dictionary with data validation."""
        # getting all class fields
        all_fields = {field.name: field.type for field in fields(cls)}
        cleaned_params = {}
        for param in params.items():
            key, value = param
            # checking if input param is in fields
            if key in all_fields:
                # checking if value type is correct
                # bool is a subclass of int
                # print(value, type(value), all_fields[key])
                if type(value) is bool:
                    if type(value) is all_fields[key]:
                        cleaned_params[key] = value
                else:
                    if isinstance(value, all_fields[key]):
                        cleaned_params[key] = value
        return cls(**cleaned_params)

    @classmethod
    def validate(cls, params):
        instance = cls.from_dict(**params)
        return instance.to_dict()



@dataclass
class AnalysisConfig(BaseDataClass):
    window_size: int
    limit_analysis_to_frame_interval: bool
    start_frame_idx: int
    end_frame_idx: int
    signal_average_threshold: float
    signal_to_noise_ratio: float
    octaves_ridge_needs_to_spann: float
    noise_window_size: int
    baseline_estimation_method: str
    include_variance: bool
    variance: int

@dataclass
class ResultsConfig(BaseDataClass):
    save_overview_png: bool
    save_detailed_results: bool
    save_single_trace_results: bool
    minimum_activity_counts: int
    signal_average_threshold: float
    signal_to_noise_ratio: float

@dataclass
class AnalysisJobConfig(BaseDataClass):
    roi_mode: str
    batch_mode: bool
    focus_area_enabled: bool
    data_source_path: Path


@dataclass
class Peak(BaseDataClass):
    frame_idx: int
    intensity: float
    amplitude: float | None = None
    delta_f_over_f: float | None = None
    has_neighboring_intersections: bool | None = None
    frame_idxs_of_neighboring_intersections: tuple | None = None
    area_under_curve: float | None = None
    peak_type: str | None = None

In [ ]:
#| export
#| hide
from neuralactivitycubic.view import WidgetsInterface

correct_general_config = WidgetsInterface().export_user_settings()
correct_general_config['save_single_trace_results'] = True  # needs to be added here until implemented in GUI

recording_filepath = Path('../test_data/00/spiking_neuron.avi')
correct_analysis_job_config = {
    'roi_mode': 'grid',
    'batch_mode': True,
    'focus_area_enabled': True,
    'data_source_path': recording_filepath,
}
correct_peak_config = {
    'frame_idx': 10,
    'intensity': 10.0,
    'amplitude': 10.0,
    'delta_f_over_f': 10.0,
    'has_neighboring_intersections': True,
    'frame_idxs_of_neighboring_intersections': (1,2),
    'area_under_curve': 10.0,
    'peak_type': 'normal',
}
minimal_peak_config = {
    'frame_idx': 10,
    'intensity': 10.0,
}

def test_correct_analysis_config():
    return AnalysisConfig.from_dict(**correct_general_config)

def test_correct_analysis_job_config():
    return AnalysisJobConfig.from_dict(**correct_analysis_job_config)

def test_correct_results_config():
    return ResultsConfig.from_dict(**correct_general_config)

def test_correct_peak_config():
    return Peak.from_dict(**correct_peak_config)

def test_minimal_peak_config():
    return Peak.from_dict(**minimal_peak_config)

incomplete_analysis_config = correct_general_config.copy()
incomplete_analysis_config.pop('window_size')

def test_incomplete_analysis_config():
    return AnalysisConfig.from_dict(**incomplete_analysis_config)

wrong_analysis_config = correct_general_config.copy()
wrong_analysis_config['window_size'] = 'haha'

def test_wrong_analysis_config():
    return AnalysisConfig.from_dict(**wrong_analysis_config)

incomplete_results_config = correct_general_config.copy()
incomplete_results_config.pop('signal_to_noise_ratio')

def test_incomplete_results_config():
    return ResultsConfig.from_dict(**incomplete_results_config)

wrong_results_config = correct_general_config.copy()
wrong_results_config['signal_to_noise_ratio'] = True

def test_wrong_results_config():
    return ResultsConfig.from_dict(**wrong_results_config)

incomplete_peak_config = correct_peak_config.copy()
incomplete_peak_config.pop('frame_idx')

def test_incomplete_peak_config():
    return Peak.from_dict(**incomplete_peak_config)

wrong_peak_config = correct_peak_config.copy()
wrong_peak_config['frame_idx'] = False

def test_wrong_peak_config():
    return Peak.from_dict(**wrong_peak_config)

In [ ]:
#| hide

# correct inputs tests
assert isinstance(test_correct_analysis_config(), AnalysisConfig)
assert isinstance(test_correct_analysis_job_config(), AnalysisJobConfig)
assert isinstance(test_correct_results_config(), ResultsConfig)
assert isinstance(test_correct_peak_config(), Peak)
assert isinstance(test_minimal_peak_config(), Peak)

# incomplete inputs tests
test_fail(test_incomplete_analysis_config)
test_fail(test_incomplete_results_config)
test_fail(test_incomplete_peak_config)

# wrong inputs tests
test_fail(test_wrong_analysis_config)
test_fail(test_wrong_results_config)
test_fail(test_wrong_peak_config)